In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


`nn.Flatten()` 和 `tensor.squeeze()` 看起来有些相似，但它们的作用实际上是不同的。

### 1. `nn.Flatten()`
- **作用**：`nn.Flatten()` 主要用于将输入的多维张量展平为一维张量。在图像分类的场景中，它通常用于将图像的二维数据（例如28x28的图像）展平为一维向量（784维）。
- **例子**：如果输入是形状为 `(batch_size, channels, height, width)` 的张量，`nn.Flatten()` 会将每个样本展平成形状为 `(batch_size, channels * height * width)` 的二维张量，通常用于连接到全连接层。

例如：
```python
x = torch.randn(32, 1, 28, 28)  # 假设输入是32张28x28的灰度图
flatten = nn.Flatten()
x_flattened = flatten(x)
print(x_flattened.shape)  # 输出：torch.Size([32, 784])
```

### 2. `tensor.squeeze()`
- **作用**：`tensor.squeeze()` 是用于去掉张量中形状为1的维度。它不会改变张量的其他维度，只会移除单维度（即尺寸为1的维度）。这个操作与展平多维张量不同，`squeeze()` 只是去除维度为1的部分。
- **例子**：如果输入的张量形状是 `(batch_size, 1, height, width)`，调用 `squeeze()` 后，它会去掉第二个维度（`1`），得到形状为 `(batch_size, height, width)` 的张量。

例如：
```python
x = torch.randn(32, 1, 28, 28)  # 输入是32张28x28的灰度图
x_squeezed = x.squeeze()
print(x_squeezed.shape)  # 输出：torch.Size([32, 28, 28])
```

### 区别：
- `nn.Flatten()` 是将输入的多维数据展平成一维向量，它是改变数据形状的方式之一，常用于处理全连接层的输入。
- `tensor.squeeze()` 则只是去除维度为 `1` 的维度，不会改变其他维度的大小。

### 举个例子：

假设输入是形状 `(32, 1, 28, 28)` 的张量：

- 使用 `nn.Flatten()`：
  ```python
  flatten = nn.Flatten()
  x_flattened = flatten(x)  # 输出的形状是 torch.Size([32, 784])
  ```

- 使用 `squeeze()`：
  ```python
  x_squeezed = x.squeeze()  # 输出的形状是 torch.Size([32, 28, 28])
  ```

`nn.Flatten()` 主要是为了将数据展平，而 `squeeze()` 只是去除单一维度。

**全连接层（Fully Connected Layer, FC Layer）** 是神经网络中一种常见的层结构，通常用于神经网络的最后几层。它的主要特点是每个输入节点与每个输出节点都连接，即每个输入都影响每个输出。

### 全连接层的基本概念：

1. **每个输入与每个输出相连**：
   - 在全连接层中，每个输入节点（神经元）都与每个输出节点连接，形成一个密集的连接结构。
   - 对于一个全连接层，输入是一个向量，输出是一个新的向量。输入的每个元素都被乘以一个权重，并加上一个偏置，结果经过激活函数输出。

2. **矩阵运算**：
   - 假设输入是一个向量 \( \mathbf{x} = [x_1, x_2, \dots, x_n] \)，全连接层会有一个权重矩阵 \( W \) 和一个偏置向量 \( \mathbf{b} \)。
   - 权重矩阵 \( W \) 的维度是 \( m \times n \)（其中 \( n \) 是输入的维度，\( m \) 是输出的维度），偏置向量 \( \mathbf{b} \) 的维度是 \( m \)。
   - 通过矩阵乘法和加法得到输出向量 \( \mathbf{y} \)：
     \[
     \mathbf{y} = W \mathbf{x} + \mathbf{b}
     \]
   - 这里的 \( \mathbf{y} \) 就是全连接层的输出，可以通过激活函数（例如 ReLU、Sigmoid 或 Tanh）进一步处理。

3. **结构**：
   - 一个全连接层的基本结构由两个部分组成：
     - **权重矩阵** \( W \)（每个输入和输出都连接）。
     - **偏置向量** \( \mathbf{b} \)（每个输出节点都有一个偏置）。
   
4. **激活函数**：
   - 通常，在全连接层的输出后会应用一个激活函数（如 ReLU、Sigmoid、Tanh 等）来增加非线性，使得神经网络可以学习更复杂的模式。

### 在深度神经网络中的作用：
- **特征组合与变换**：
  全连接层可以将前一层（通常是卷积层或池化层）提取的特征组合和变换，从而使网络可以学到更高级的抽象特征。它通常用于提取抽象的、非线性的组合特征，最终进行分类或回归任务。
  
- **输出层**：
  在许多网络中，最后一层是一个全连接层，它输出的是每个类别的概率值（在分类任务中）或者是预测值（在回归任务中）。

### 举个例子：

在一个图像分类的神经网络中，通常会先使用卷积层（Convolutional Layer）和池化层（Pooling Layer）提取图像的特征，然后使用全连接层将这些特征进行组合，最后输出分类结果。假设我们有一个简单的网络结构：

1. **输入**：一个 28x28 的灰度图像（例如，MNIST 数据集中的数字图片）。
2. **卷积层和池化层**：提取图像中的边缘、纹理等低级特征。
3. **全连接层**：
   - 将提取到的特征展平为一个向量（784维）。
   - 通过一个或多个全连接层进一步处理这些特征，最后输出 10 个类别的概率（对于 MNIST，类别数是 10）。
4. **输出**：类别标签（例如，0、1、2、...、9）。

### 代码中的全连接层：
在你提供的代码中，`self.linear_relu_stack` 就是由多个全连接层组成的一个堆栈：

```python
self.linear_relu_stack = nn.Sequential(
    nn.Linear(28*28, 512),  # 输入层，将28x28的图像展平后输入，输出维度为512
    nn.ReLU(),               # ReLU激活函数
    nn.Linear(512, 512),     # 第二个全连接层，输入512维，输出512维
    nn.ReLU(),               # ReLU激活函数
    nn.Linear(512, 10),      # 输出层，输出10个类别的预测值
)
```

在这里，`nn.Linear` 就表示全连接层，每个 `Linear` 层会将前一层的输出（一个向量）乘以一个权重矩阵并加上偏置，再通过激活函数（如 `ReLU`）输出结果。

是的，正如你所说，`nn.Linear` 就是全连接层，它实现了公式 \( \mathbf{y} = W\mathbf{x} + b \)，其中：
- \( \mathbf{x} \) 是输入向量（例如，展平后的图像），
- \( W \) 是权重矩阵，
- \( b \) 是偏置项，
- \( \mathbf{y} \) 是输出向量。

`nn.Linear` 会对输入进行加权求和，然后加上偏置，再通过激活函数（如 `ReLU`）得到输出。

### 1. **`nn.Linear`（全连接层）**:
- **工作方式**：每个输入元素与每个输出元素都相连。
- **公式**：\( \mathbf{y} = W\mathbf{x} + b \)
- **用途**：将输入特征进行线性组合并输出，通常用于神经网络的最终阶段，进行决策或预测。

### 2. **`nn.Conv2d`（卷积层）**:
`nn.Conv2d` 是卷积神经网络（CNN）中常用的层，它用于提取图像的局部特征，不同于全连接层，卷积层有以下特点：

- **工作方式**：卷积层通过小的滤波器（或称卷积核）扫描输入图像，并在每个位置上计算局部的加权和。
- **公式**：卷积操作可以表示为：
  \[
  \mathbf{y}(i,j) = \sum_k \mathbf{w}(k) \cdot \mathbf{x}(i+k,j+k) + b
  \]
  其中 \( \mathbf{w} \) 是卷积核，\( \mathbf{x} \) 是输入图像，\( \mathbf{y} \) 是卷积输出。
- **特点**：卷积层的权重是共享的，即同一卷积核在整个输入图像上滑动，进行特征提取。这使得卷积层的参数数量比全连接层要少得多。
- **用途**：卷积层通常用于提取局部特征（例如边缘、纹理、颜色等），它是卷积神经网络（CNN）中非常重要的一部分，广泛用于图像处理、计算机视觉等任务。

### 差异：
- **全连接层（`nn.Linear`）**：每个输入元素都与每个输出元素相连，适用于对所有特征进行全局处理（通常用于分类和回归任务的最后阶段）。
- **卷积层（`nn.Conv2d`）**：通过局部感知的卷积核扫描输入图像，适用于提取局部特征（用于图像分类、目标检测等任务）。

### 举个例子：
假设我们有一个 28x28 的输入图像（比如 MNIST 手写数字数据集）：
- **全连接层**：将 28x28 的图像展平为一个 784 维的向量，然后通过 `nn.Linear(784, 10)` 进行分类。
  
  这时每个像素值都会对最终的分类结果产生影响。

- **卷积层**：如果使用 `nn.Conv2d(1, 32, kernel_size=3)`，它会使用 32 个大小为 3x3 的卷积核，在图像上滑动，提取局部特征（如边缘、角点、纹理等），输出一个 32 通道的特征图。这里每个卷积核只会处理图像的局部区域，权重是共享的。

### 总结：
- `nn.Linear` 是全连接层，通常用于神经网络的后部，进行高维的特征融合和输出决策。
- `nn.Conv2d` 是卷积层，通常用于神经网络的前部，用于提取图像中的局部特征。

In [13]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")


Predicted class: tensor([1], device='cuda:0')


确实，这个情况有些奇怪，因为在你给出的代码中，模型的 `forward()` 方法应该依赖于输入数据 `X`，但是在你描述的情况中，模型似乎没有显式地训练或接受输入数据，而直接就能进行预测。让我们一步步分析一下：

### 代码分析：

```python
X = torch.rand(1, 28, 28, device=device)  # 生成一个随机的张量作为输入
logits = model(X)  # 使用模型进行前向传播，得到logits（预测结果的未归一化分数）
pred_probab = nn.Softmax(dim=1)(logits)  # 使用Softmax得到每个类别的概率分布
y_pred = pred_probab.argmax(1)  # 获取最大概率对应的类别
print(f"Predicted class: {y_pred}")  # 输出预测类别
```

### 关键点：
- 你使用了 `torch.rand(1, 28, 28, device=device)` 来生成一个随机张量 `X` 作为输入。这个张量形状为 `(1, 28, 28)`，它就代表了一个批次（batch）中包含的一张 28x28 的图像。
- `model(X)` 将这个输入张量传入到模型中，调用了 `forward()` 方法，进行前向传播并计算 `logits`。
- 使用 `Softmax` 来计算预测的类别概率。
- `argmax(1)` 会返回概率最大的类别索引，作为模型的预测结果。

### 为什么能预测？
即使你没有对模型进行显式的训练，`model(X)` 仍然可以给出一个预测，这是因为：

1. **模型结构已经定义**：即使模型没有进行训练，模型结构（如层的定义）已经存在。模型的 `forward()` 方法会按照定义的结构（即 `Flatten` 和 `Linear` 层）对输入数据进行处理。
   
2. **没有训练的数据不影响模型运行**：模型的每个层（如 `Linear` 层）都会有默认的权重和偏置（这些默认的值通常是随机初始化的）。所以，当你调用 `model(X)` 时，模型会使用这些随机初始化的权重进行前向传播计算，生成一个 `logits`（即网络输出的未归一化的预测分数）。

3. **预测不需要训练**：即使没有训练，网络的每一层都能计算出一个输出，虽然这个输出没有经过训练调整，通常是不准确的，但它仍然能够进行“预测”。这是因为模型结构和计算过程是已经定义好的。

### 为什么输出会有一个预测类别？
- 由于权重和偏置是随机初始化的，`model(X)` 会计算出一个输出（可能非常随机、不准确）。
- `Softmax` 会将这个输出的 logits 转换为概率分布，`argmax(1)` 则会选出概率最大的类别作为模型的预测。
- 所以，即使模型没有经过训练，它也能基于随机初始化的参数输出一个“预测”结果。这个结果并不是有意义的，因为没有训练，模型还没有学习到任何有用的模式。

### 总结：
- **模型能预测**：因为模型已经有结构和参数（随机初始化），即使没有训练，`model(X)` 依然能够进行计算，产生一个预测输出。
- **为什么不准确**：这些预测不准确，因为模型的权重是随机初始化的，尚未经过训练。

如果你希望模型的预测有实际意义，必须要对它进行训练，使用真实的数据来更新模型的参数。

In [15]:
input_image = torch.rand(3,28*28)
print(input_image.size())

torch.Size([3, 784])


In [16]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
layer1 = nn.Linear(in_features=28*28,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [17]:
print(f"Model structure: {model}\n")
for name,param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0196,  0.0178,  0.0279,  ..., -0.0180, -0.0160,  0.0288],
        [-0.0239,  0.0111,  0.0235,  ...,  0.0098, -0.0083, -0.0158]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0224,  0.0253], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0285, -0.0408,  0.0294,  ...,  0.0339, -0.0080, -0.0204],
        [ 0.0340,  0.0132,  0.0260,  ...,  0.0348,  0.0101,  0.0091]],
       device='cuda:0', grad_fn=<SliceB

There are reasons you might want to disable gradient tracking:
To mark some parameters in your neural network as frozen parameters.

To speed up computations when you are only doing forward pass, because computations on tensors that do not track gradients would be more efficient.
- method 1  
```python
z = torch.matmul(x, w)+b  
print(z.requires_grad)  

with torch.no_grad():  
    z = torch.matmul(x, w)+b  
print(z.requires_grad)  
```

- method2  
```python
z = torch.matmul(x, w)+b  
z_det = z.detach()  
print(z_det.requires_grad)
```  

是的，**计算图（computation graph）** 就是指在深度学习中执行前向传播（forward）、损失计算（loss）、反向传播（backpropagation）和优化（optimization）等一系列操作的过程，它们通常通过一个有向图的形式表示。

在深度学习中，计算图用于描述一个数学表达式的计算过程，每个节点代表一个操作或张量，而每条边则表示数据（张量）的流动。

### 计算图的主要部分包括：

1. **前向传播（Forward Pass）**：
   - 在前向传播过程中，输入数据经过网络的各个层进行处理，逐步生成最终的输出（例如预测结果）。
   - 在计算图中，前向传播的每个操作（如矩阵乘法、加法、激活函数等）都会被表示为一个节点，每个节点都会接收来自前一节点的输出。
   - **举例**：对于一个神经网络，输入层接收到输入数据后，经过每个隐藏层的操作（例如线性变换和激活函数），最终输出预测值。

2. **损失计算（Loss Calculation）**：
   - 前向传播完成后，计算损失函数（如交叉熵损失、均方误差损失等），损失函数计算的是模型预测值与真实值之间的差距。
   - 在计算图中，损失函数是一个特殊的节点，它依赖于前向传播的输出和真实标签（ground truth）。

3. **反向传播（Backpropagation）**：
   - 反向传播的目的是通过链式法则来计算每个参数的梯度。这个过程会从损失节点开始，向后传递梯度，并更新网络中的参数。
   - 反向传播过程涉及计算每个节点（操作）对最终损失的贡献，并依此更新参数（例如权重和偏置）。
   - 在计算图中，反向传播是从损失节点开始，沿着图的反向方向计算梯度。通过链式法则，计算梯度并将其传递回每个层，直到输入层。

4. **优化（Optimization）**：
   - 使用反向传播得到的梯度来更新模型参数（权重和偏置），以减小损失函数的值。
   - 优化方法（如梯度下降、Adam 等）会根据梯度信息调整模型的参数，使得模型在训练集上表现得更好。

### 计算图的工作流程（以神经网络为例）：

1. **输入数据**进入网络。
2. **前向传播**计算每一层的输出（包括线性变换、激活等操作）。
3. **损失计算**：根据网络输出与真实标签计算损失。
4. **反向传播**：计算每个参数的梯度。
5. **优化**：通过优化算法（如梯度下降）使用反向传播计算得到的梯度来更新参数。

### 计算图的实现：

深度学习框架如 TensorFlow 和 PyTorch 都使用计算图来管理前向传播和反向传播过程。

- **TensorFlow**：计算图是静态的，需要在执行之前先构建图并定义操作。然后，通过会话（session）来执行计算图。
- **PyTorch**：计算图是动态的，每次前向传播都会即时构建计算图（称为“动态图”）。在每次前向传播时，PyTorch 会根据需要即时构建计算图并执行。

### 举个 PyTorch 例子：

在 PyTorch 中，计算图是动态构建的。当你执行前向传播时，PyTorch 会自动记录操作，并在反向传播时计算梯度。

```python
import torch
import torch.nn as nn
import torch.optim as optim

# 简单的线性回归模型
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

# 数据
X = torch.tensor([[1.0], [2.0], [3.0], [4.0]])  # 输入数据
y = torch.tensor([[2.0], [4.0], [6.0], [8.0]])  # 真实标签

# 模型、损失函数和优化器
model = SimpleNN()
criterion = nn.MSELoss()  # 均方误差损失
optimizer = optim.SGD(model.parameters(), lr=0.01)  # 使用SGD优化器

# 训练过程
for epoch in range(100):
    # 前向传播
    y_pred = model(X)
    
    # 计算损失
    loss = criterion(y_pred, y)
    
    # 反向传播
    optimizer.zero_grad()  # 清零之前的梯度
    loss.backward()  # 计算梯度
    
    # 优化
    optimizer.step()  # 更新参数
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')
```

在这个例子中，PyTorch 会根据 `model(X)` 动态构建计算图，记录前向传播的操作，然后在 `loss.backward()` 时计算梯度，最后在 `optimizer.step()` 时更新模型参数。

### 总结：
计算图是深度学习中用于表示和执行神经网络操作的一个抽象，它表示了前向传播、损失计算、反向传播和优化过程中的所有操作。前向传播计算输出，损失函数计算误差，反向传播计算梯度，优化步骤根据梯度更新网络参数。